In [ ]:
from random import choice as RD1
from random import sample as RD2
from datetime import datetime
import requests
import base64
import time
import csv
import os

batch_name = "batch003"
os.makedirs(f"output/{batch_name}", exist_ok=True)

In [ ]:
def generate_prompt():
    return ", ".join([e for e in [
    RD1(['', 'candid photo', 'amateur photo']),
    RD1(['a man', 'a young man', 'a 25 years old man']),
    RD1(['toned body', 'fit body', 'defined body', 'athletic body', 'solid body', 'muscular body']),

    # RD1(['', 'asian skin']),
    RD1(['', 'asian', 'asian skin', 'asian skin', 'asian skin']),

    # wearing
    RD1([
        # RD1(['black suit, black pants']),
        # RD1(['black dress shirt, black pants', 'black dress shirt, black pants, black belt']),
        # RD1(['black tight shirt, grey baggy pants', 'black tight shirt, grey loose sweatpants']),
        RD1(['topless, grey loose sweatpants', 'topless, grey baggy pants']),
        # RD1(['topless, grey tight boxer', 'topless, grey tight shorts']),
        # 'sexy, '+RD1(['penis', 'dick', 'cock', 'big penis', 'big dick', 'big cock', 'huge penis', 'huge dick', 'huge cock']),
        # 'nude, '+RD1(['penis', 'dick', 'cock', 'big penis', 'big dick', 'big cock', 'huge penis', 'huge dick', 'huge cock']),
    ]),

    RD1(['taking mirror selfie', 'taking a mirror selfie']),
    RD1(['plain black wall', 'plain dark wall']),
    RD1(['soft lighting', 'dim lighting', 'low lighting']),
    RD1(['', 'film grain']),
    RD1(['', 'dark aesthetic', 'dark vibe', 'dark mafia vibe', 'dark romance vibe']),
    ] if e != ''])
generate_prompt()

In [ ]:
payload = {"alwayson_scripts": {}, "batch_size": 1, "comments": {}, "denoising_strength": 0.7, "disable_extra_networks": False, "distilled_cfg_scale": 3.5, "do_not_save_grid": False, "do_not_save_samples": False, "enable_hr": False, "hr_additional_modules": ["Use same choices"], "hr_cfg": 5, "hr_distilled_cfg": 3.5, "hr_negative_prompt": "", "hr_prompt": "", "hr_resize_x": 0, "hr_resize_y": 0, "hr_scale": 2, "hr_second_pass_steps": 0, "hr_upscaler": "Latent", "n_iter": 1, "override_settings_restore_afterwards": False, "prompt": "candid photo, amateur photo, iphone selfie, sexy man, abs, penis, shirt lift, dim dark room, low light, film grain", "restore_faces": False, "s_churn": 0.0, "s_min_uncond": 0.0, "s_noise": 1.0, "s_tmax": 0, "s_tmin": 0.0, "script_args": [], "script_name": "", "seed": 2812549172, "seed_enable_extras": True, "seed_resize_from_h": -1, "seed_resize_from_w": -1, "steps": 25, "styles": [], "subseed": -1, "subseed_strength": 0, "tiling": False, "negative_prompt": "", "width": 896, "height": 1152, "cfg_scale": 2.5, "sampler_name": "DPM++ 2M SDE", "scheduler": "Karras", "override_settings": {"sd_model_checkpoint": "LVSTIFY_V5.0", "CLIP_stop_at_last_layers": 1}}

In [ ]:
error_count = 0

generations = []
for _ in range(500):
    try:
        payload["prompt"] = generate_prompt()
        payload["seed"] = -1
        payload["cfg_scale"] = RD1([2.5, 3.0, 3.5])
        payload["steps"] = RD1([20, 25, 30])
        filename = f"output/{batch_name}/{datetime.now().strftime('%y%m%d%H%M%S')}.jpg"
        generations.append({
            "filename": filename,
            "prompt": payload["prompt"],
            "cfg_scale": payload["cfg_scale"],
            "steps": payload["steps"],
        })
        with requests.post(url="http://127.0.0.1:7860/sdapi/v1/txt2img", json=payload) as req:
            with open(filename, "wb") as f:
                f.write(base64.b64decode(req.json()["images"][0]))
            with open(f"output/{batch_name}.csv", mode='w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=generations[0].keys())
                writer.writeheader()
                writer.writerows(generations)
            time.sleep(10)
    except Exception as e:
        print(f"Error > {e}")
        error_count += 1
        if error_count == 10:
            break